# 01 — Polars: expressões + Lazy (performance)

Objetivo: reescrever análises do Pandas com Polars, focando em performance e legibilidade.

Tempo: ~25–30 min

In [ ]:
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    """Sobe diretórios até encontrar uma 'marca' do repositório (README.md + pasta data)."""
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / "README.md").exists() and (cur / "data").exists():
            return cur
        cur = cur.parent
    # fallback: assume cwd
    return Path.cwd().resolve()

ROOT = find_repo_root()
DATA_DIR = ROOT / "data"
SAMPLE_DIR = DATA_DIR / "sample"

print("ROOT:", ROOT)
print("SAMPLE_DIR:", SAMPLE_DIR)

## 1) Lendo CSV com Polars

Polars é rápido e tem uma API baseada em *expressions*.

In [ ]:
import polars as pl

sales_pl = pl.read_csv(SAMPLE_DIR / "sales.csv")
sales_pl.head()

## 2) Expressions

Crie colunas, filtre, agregue — sem loops.

In [ ]:
sales_pl = sales_pl.with_columns(
    (pl.col("qty") * pl.col("unit_price")).alias("revenue_calc")
)

by_region = (
    sales_pl.group_by("region")
            .agg([
                pl.col("revenue_calc").sum().alias("revenue_total"),
                pl.col("order_id").n_unique().alias("pedidos"),
            ])
            .sort("revenue_total", descending=True)
)

by_region

## 3) Lazy (scan + otimização)

Mesmo com CSV pequeno, vale aprender o padrão.

In [ ]:
lazy = pl.scan_csv(SAMPLE_DIR / "sales.csv")

result = (
    lazy.with_columns((pl.col("qty") * pl.col("unit_price")).alias("revenue_calc"))
        .group_by(["region","category"])
        .agg(pl.col("revenue_calc").sum().alias("revenue_total"))
        .sort("revenue_total", descending=True)
        .collect()
)

result.head(10)

## Exercícios

1- Faça top 5 produtos por receita.
2- Compare tempo `read_csv` vs `scan_csv().collect()` (use `%time` no notebook).
3- Salve o resultado em Parquet (use `result.write_parquet`).

In [ ]:
# Escreva suas respostas aqui
